In [1]:
from cases import *

# case = Debugger3
cases = [case_1, case_2, case_3, case_4, case_5]


In [2]:
import numpy as np
from time import time 
import math

np.random.seed(int(time()))
EPS = 1

class Gene:
    fitness = 0
    def __init__(self, perm) -> None:
        self.perm = perm

class SA:
    def __init__(self, case: Case, T, alpha, mcn, mgni) -> None:
        self.case = case
        self.state = Gene(case.req.copy())
        np.random.shuffle(self.state.perm)
        self.state.fitness = self.calculate_fitness(self.state)
        self.fittest = self.state

        self.T = T
        self.alpha = alpha
        self.N = mcn
        self.n = 0

        self.gen = 0
        self.mgni = mgni # Max Generations Without Improvement
        self.terminated = False

    def iterate(self):
        if self.gen == self.mgni:
            self.terminated = True
            return

        if self.n == self.N:
            self.T = self.new_T(self.T)
            self.n = 0

        self.n += 1
        self.gen += 1

        neighbour = self.get_random_neighbour(self.state)
        fitness = self.calculate_fitness(neighbour)
        
        neighbour.fitness = fitness
        delta = fitness - self.state.fitness

        if delta < 0:
            self.change_state(neighbour)
            return
        
        prob = self.prob_func(delta, self.T)
        unif = np.random.random()

        if unif >= prob:
            self.change_state(neighbour)

    def calculate_fitness(self, state):
        L = self.case.L
        result = 0
        curr_log = 0
        loss = {}

        for order in state.perm:
            if order > curr_log:
                if curr_log not in loss:
                    loss[curr_log] = 0
                loss[curr_log] += 1
                curr_log = L
                result += 1
            mn = min([1_000_000_000] + [i for i in loss if i >= order])
            if mn == 1_000_000_000 or loss[mn] <= 0:
                curr_log -= order
            else:
                loss[mn] -= 1
                new_loss = mn - order
                if new_loss not in loss:
                    loss[new_loss] = 0

                loss[new_loss] += 1

        if curr_log == L:
            result -= 1

        return result

    def get_random_neighbour(self, state: Gene):
        pos1, pos2 = 0, 0
        while pos1 == pos2:
            pos1 = np.random.randint(0, len(state.perm))
            pos2 = np.random.randint(0, len(state.perm))

        result = Gene(state.perm.copy())
        result.perm[pos1], result.perm[pos2] = result.perm[pos2], result.perm[pos1]

        return result

    def change_state(self, new_state):
        if new_state.fitness - self.state.fitness <= EPS:
            self.gen = 0
            self.fittest = new_state
        self.state = new_state

    def new_T(self, old_T):
        return self.alpha * old_T

    def prob_func(self, d, T):
        return math.pow(math.e, -float(d)/10.0/float(T))


In [7]:
T = 10000
alpha = 0.97
markov_chain_length = 200
instances = 50
problems = {}

for case in cases:
    for instance in range(instances):
        if case.name not in problems:
            problems[case.name] = []

        problem = SA(case, T, alpha, markov_chain_length, 20 * markov_chain_length)
        problems[case.name].append(problem)

done = False

while not done:
    flag = True
    for key in problems:
        for problem in problems[key]:
            problem.iterate()
            flag = flag & problem.terminated

    if not flag:
        done = True


In [8]:
best_solutions = {}

for case in problems:
    for problem in problems[case]:
        if case not in best_solutions:
            best_solutions[case] = problem.fittest
        
        if best_solutions[case].fitness > problem.fittest.fitness:
            best_solutions[case] = problem.fittest

for case in best_solutions:
    print(f"best answer found for {case} is {best_solutions[case].fitness}")
    

best answer found for case 1 is 52
best answer found for case 2 is 81
best answer found for case 3 is 98
best answer found for case 4 is 210
best answer found for case 5 is 3909
